# QTA - Prepare Data

## Import Libraries

In [1]:
# General imports
import pandas as pd
import os
import time
import re

Data is stored in .XML files which we need to parse.

In [ ]:
# Libraries for parsing data
from lxml import etree
from bs4 import BeautifulSoup

 Additionally, we employ multiprocessing for faster execution.

In [ ]:
# Libraries for multiprocessing
import multiprocessing as mp
from multiprocessing import Pool

# Check core count
num_cores = mp.cpu_count()
print(num_cores)

## Load Text Data

We define the data to load. We could set a specific sample to only load a certain amount of articles for fast code testing. However, ultimately we want to load all the articles in our data set.

In [ ]:
# Set corpus to the folder of files you want to use
corpus = '/home/ec2-user/SageMaker/data/QTA_-_titles/'

# Read in files
input_files = os.listdir(corpus)

# Select the number of articles to sample
sample_input_files = input_files# [0:50000] # len(input_files)
    
print("Currently sampling", len(sample_input_files), "documents.")

We define the function to parse the .XML tree. We also include the function for strip html text. (These functions were part of the example script from TDM Studio).

In [ ]:
# Function to strip html tags from text portion
def strip_html_tags(text):
    stripped = BeautifulSoup(text).get_text().replace('\n', ' ').replace('\\', '').strip()
    return stripped


# Retrieve metadata from XML document
def getxmlcontent(corpus, file, strip_html=True):
    try:
        tree = etree.parse(corpus + file)
        root = tree.getroot()

        if root.find('.//GOID') is not None:
            goid = root.find('.//GOID').text
        else:
            goid = None

        if root.find('.//Title') is not None:
            title = root.find('.//Title').text
        else:
            title = None

        if root.find('.//NumericDate') is not None:
            date = root.find('.//NumericDate').text
        else:
            date = None
            
        if root.find('.//PublisherName') is not None:
            publisher = root.find('.//PublisherName').text
        else:
            publisher = None

        if root.find('.//FullText') is not None:
            text = root.find('.//FullText').text

        elif root.find('.//HiddenText') is not None:
            text = root.find('.//HiddenText').text

        elif root.find('.//Text') is not None:
            text = root.find('.//Text').text

        else:
            text = None

        # Strip html from text portion
        if text is not None and strip_html == True:
            text = strip_html_tags(text)
    
    except Exception as e:
        print(f"Error while parsing file {file}: {e}")
    
    return goid, title, date, publisher, text


# Function to make lists out of parsed data--on single document scale for multiprocessing
def make_lists(file):
    
    goid, title, date, publisher, text = getxmlcontent(corpus, file, strip_html=True)
    
    return goid, publisher, title, text, date

We test the function on a single document before we apply it through multiprocessing to all documents.

In [ ]:
# Test function on single document
make_lists(input_files[0])

In [ ]:
start_time = time.time()
try:
    p = Pool(processes=num_cores-1)
    processed_lists = p.map(make_lists, sample_input_files)
except Exception as e:
    print(f"Error in processing document: {e}")
finally:
    p.close()
    
df = pd.DataFrame(processed_lists, columns=['GOID', 'Publisher', 'Title', 'Text', 'Date'])
end_time = time.time()

print("Execution time: ", end_time-start_time)

Finally, we have a dataframe that includes everything that we read in (i.e. GOID, Publisher, Title, Text, Date) for each of our articles.

In [ ]:
# Sort for reproducibility
df_sorted = df.sort_values(by = ["Date", "Title"])
df = df_sorted.reset_index(drop=True)
df

In [ ]:
# View dataframe
print(df.iloc[0])
print()

# View full example
print(df.iloc[0]["Title"])
print()
print(print(df.iloc[0]["Text"]))

## Filter data based on company name

The goal is to filter articles down in multiple iterations, because each simple iteration makes the next filter less computationally expensive.

First, we filter out the articles that contain a company name in the title and the articles that contain a company name in the text.

In [2]:
company_df = pd.read_csv("Companies.csv", encoding="cp1252")
company_df = company_df[company_df["Companies"] != "Target"]
company_df = company_df[company_df["Companies"] != "Visa"]

comp_df = company_df["Companies"].to_list()
pattern_comp = re.compile(r'\b(?:' + '|'.join(re.escape(company) for company in comp_df) + r')\b')


def contains_company_name(text, pattern):
    if text:        
        return bool(re.search(pattern, text))
    else:
        return False

In [3]:
pattern_comp

re.compile(r'\b(?:Exxon\ Mobil|Wal\-Mart\ Stores|General\ Motors|Ford\ Motor|General\ Electric|Citigroup|Enron|Intl\.\ Business\ Machines|AT\&T|Verizon\ Communications|Altria\ Group|Chase\ Manhattan\ Corp\.|Bank\ of\ America\ Corp\.|SBC\ Communications|Boeing|Texaco|Duke\ Energy|Kroger|Hewlett\-Packard|ChevronTexaco|State\ Farm\ Insurance\ Cos|American\ Intl\.\ Group|Home\ Depot|Morgan\ Stanley|Merrill\ Lynch|Fannie\ Mae|Compaq\ Computer|Lucent\ Technologies|Sears\ Roebuck|Merck|Procter\ \&\ Gamble|MCI\ WorldCom|TIAA\-CREF|Motorola|McKesson|Kmart\ Holding|Albertson\'s|Marathon\ Oil|Berkshire\ Hathaway|Intel|Goldman\ Sachs\ Group|J\.C\.\ Penney|Conoco|Costco\ Wholesale|Safeway|MetLife|Dell|Ingram\ Micro|Freddie\ Mac|Cardinal\ Health|United\ Parcel\ Service|Pfizer|Dynegy|CenterPoint\ Energy|DuPont|Johnson\ \&\ Johnson|Delphi|Allstate|Aquila|International\ Paper|Wells\ Fargo|Aetna|United\ Technologies|Lehman\ Brothers\ Hldgs\.|BellSouth|Walt\ Disney|ConAgra\ Foods|Lockheed\ Martin|Bank\ O

In [ ]:
start_time = time.time()
title_df = df[df["Title"].apply(lambda x: contains_company_name(x, pattern_comp))]
end_time = time.time()
print("Execution time: ", end_time-start_time)
title_df.to_csv("QTATITLE/filtered_df_title.csv")

In [ ]:
start_time = time.time()
text_df = df[df["Text"].apply(lambda x: contains_company_name(x, pattern_comp))]
end_time = time.time()
print("Execution time: ", end_time-start_time)
text_df.to_csv("QTATITLE/filtered_df_text.csv")

## Filter by Keywords

Second, we filter out the articles that contain a keyword. Here, we distinguish again between contained in the title or contained in the text.

In [4]:
scandal_df = pd.read_csv("keywords.csv", index_col=False)
scand_df = scandal_df["keywords"].to_list()
pattern_string = '|'.join(re.escape(sca) for sca in scand_df)
pattern_string_repl = pattern_string.replace("\.\*", ".*")
pattern_scandal = re.compile(r'\b(?:' + pattern_string_repl + r')\b', re.IGNORECASE)


def contains_scandal(text, pattern):
    if text:        
        return bool(re.search(pattern, text))
    else:
        return False

In [5]:
pattern_scandal

re.compile(r'\b(?:corrupt.*|deceit.*|decept.*|deceiv.*|betray.*|shame.*|scandal.*|dishonest.*|misconduct.*|fraud|illegal.*|unethic.*|violat.*|falsify.*|breach.*|leak.*|pollut.*|insecur.*|irregular.*|mismanag.*|inappropriat.*|unlaw.*|transgress.*|noncomplian.*|non\-complian.*|ghost.*|malfeas.*|exploitat.*|discriminat.*|harass.*|misrepresent.*|embezzle.*|improper.*|espion.*|collus.*|misus.*|rigg.*|kickback.*|retaliat.*|moral\ lapse|insider\ trading|insider\ dealing|ponzi\ scheme|arrest.*|product\ recall|privacy\ breach|privacy\ violation\*|data\ leak|intellectual\ property\ dispute|malpracti.*|destruct.*|unsustain.*|ESG\ scandal|fraudulent|deceptiv.*|scamm.*|briber.*|bribe.*|extort.*|misappropriat.*|sabotag.*|deforest.*|habitat\ destruct.*|climate\ change\ deni.*|tax\ evasion|money\ launder.*|accounting\ scandal|whistleblow.*|sexual\ harass.*|workplace\ harass.*|toxic\ culture|data\ breach|ransomware|drug\ recall|clinical\ trial\ fraud|off\-label\ marketing|antitrust|cartel|monopoly|liti

### "Title" Data

This filters for scandal words contained in the title.

In [ ]:
loaded_title_df = pd.read_csv("QTATITLE/filtered_df_title.csv", index_col=0)

In [ ]:
start_time = time.time()
title_df_scand = loaded_title_df[loaded_title_df["Title"].apply(lambda x: contains_scandal(x, pattern_scandal))]
end_time = time.time()
print("Execution time: ", end_time-start_time)
title_df_scand.to_csv("QTATITLE/scand_df_title.csv") # Execution time:  3.7135045528411865

In [ ]:
title_df_scand

### "Text" Data

This filters for scandal words contained in the text.

In [ ]:
loaded_text_df = pd.read_csv("QTATITLE/filtered_df_text.csv", index_col=0)

In [ ]:
start_time = time.time()
text_df_scand = loaded_text_df[loaded_text_df["Text"].apply(lambda x: contains_scandal(x, pattern_scandal))]
end_time = time.time()
print("Execution time: ", end_time-start_time)
text_df_scand.to_csv("QTATITLE/scand_df_text.csv") # Execution time:  726.7273228168488

In [ ]:
text_df_scand

In [ ]:
loaded_text_df_scand = pd.read_csv("QTATITLE/scand_df_text.csv", index_col=0)
loaded_text_df_scand

We then filter again, such that we only keep an article if the scandal is contained within +/- 15 words (excl. stopwords) around a company name.

In [ ]:
company_df = pd.read_csv("Companies.csv", encoding="cp1252")
company_df = company_df[company_df["Companies"] != "Target"]
company_df = company_df[company_df["Companies"] != "Visa"]
comp_df = company_df["Companies"].to_list()
pattern_comp = re.compile(r'\b(?:' + '|'.join(re.escape(company) for company in comp_df) + r')\b', re.IGNORECASE)  

scandal_df = pd.read_csv("keywords.csv", index_col=False)
scand_df = scandal_df["keywords"].to_list()
pattern_string = '|'.join(re.escape(sca) for sca in scand_df)
pattern_string_repl = pattern_string.replace("\.\*", ".*")
pattern_scandal = re.compile(r'\b(?:' + pattern_string_repl + r')\b', re.IGNORECASE)

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.data.path.append("./stopwords_en")
stop_words = set(stopwords.words("english_stopwords"))

def remove_my_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return " ".join(filtered_words)

def check_keywords_context(text):
    global keywords_counter
    keywords_counter += 1
    if keywords_counter%1000==0:
        print(keywords_counter)
    filtered_text = remove_my_stopwords(text)

    for match in re.finditer(pattern_comp, filtered_text):
        company_start, company_end = match.span()
        company_index = filtered_text[:company_start].count(" ")
        
        start_index = max(0, company_index-15)
        end_index = min(len(filtered_text.split()), company_index+15)
        context_words = filtered_text.split()[start_index:end_index]
        
        context_text = " ".join(context_words)
        if pattern_scandal.search(context_text):
            return True
    
    return False     

keywords_counter = 0
start_time = time.time()
text_df_scand_context = loaded_text_df_scand[loaded_text_df_scand["Text"].apply(lambda x: check_keywords_context(x))]
end_time = time.time()
print("Execution time: ", end_time-start_time)
text_df_scand_context.to_csv("QTATITLE/scand_df_context_text.csv") # Execution time:  1729.6244366168976

In [ ]:
text_df_scand_context

## Title-Text

Finally we merge the previous results such that we know for each article why it was kept (Titlebased, Textbased, or both).

In [ ]:
loaded_title_df_scand = pd.read_csv("QTATITLE/scand_df_title.csv", index_col=0)
loaded_title_df_scand = loaded_title_df_scand[["GOID", "Title", "Text", "Date"]]
loaded_title_df_scand["Titlebased"] = 1
loaded_title_df_scand

In [ ]:
loaded_text_df_scand_context = pd.read_csv("QTATITLE/scand_df_context_text.csv", index_col=0)
loaded_text_df_scand_context = loaded_text_df_scand_context[["GOID", "Title", "Text", "Date"]]
loaded_text_df_scand_context["Textbased"] = 1
loaded_text_df_scand_context

In [ ]:
title_text_df = pd.merge(loaded_title_df_scand, loaded_text_df_scand_context, left_on="GOID", right_on="GOID", how="outer")

cleaned_title_text_df = pd.DataFrame()

for col in title_text_df.columns:
    if col.endswith("_x"):
        base_col = col[:-2]
        if base_col + "_y" in title_text_df.columns:
            cleaned_title_text_df[base_col] = title_text_df[col].combine_first(title_text_df[base_col + "_y"])
        else:
            cleaned_title_text_df[base_col] = title_text_df[col]
    elif col.endswith("_y") and col[:-2] + "_x" not in title_text_df.columns:
        cleaned_title_text_df[col[:-2]] = title_text_df[col]
    elif '_' not in col:
        cleaned_title_text_df[col] = title_text_df[col]
         
cleaned_title_text_df.to_csv("QTATITLE/cleaned_tt_df.csv")
cleaned_title_text_df

In [ ]:
loaded_tt_df = pd.read_csv("QTATITLE/cleaned_tt_df.csv", index_col=0)
loaded_tt_df["Text"] = loaded_tt_df["Text"].fillna("")
loaded_tt_df

In [ ]:
loaded_tt_df_2 = loaded_tt_df[["GOID", "Titlebased", "Textbased"]]
loaded_tt_df_2.to_csv("QTATITLE/cleaned_tt_df_download.csv")
loaded_tt_df_2

We also shorten the text, such that we only have a list of strings containing the +/-15 words around the company names).

In [ ]:
company_df = pd.read_csv("Companies.csv", encoding="cp1252")
company_df = company_df[company_df["Companies"] != "Target"]
company_df = company_df[company_df["Companies"] != "Visa"]
comp_df = company_df["Companies"].to_list()
pattern_comp = re.compile(r'\b(?:' + '|'.join(re.escape(company) for company in comp_df) + r')\b', re.IGNORECASE)  

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.data.path.append("./stopwords_en")
stop_words = set(stopwords.words("english_stopwords"))

def remove_my_stopwords(text):
    try:
        words = text.split()
    except:
        print(text)
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return " ".join(filtered_words)

def filter_keywords_context(text):
    global keywords_counter
    keywords_counter += 1
    if keywords_counter%1000==0:
        print(keywords_counter)
    filtered_text = remove_my_stopwords(text)
    
    text_filtered_out = []
    for match in re.finditer(pattern_comp, filtered_text):
        company_start, company_end = match.span()
        company_index = filtered_text[:company_start].count(" ")
        
        start_index = max(0, company_index-15)
        end_index = min(len(filtered_text.split()), company_index+15)
        context_words = filtered_text.split()[start_index:end_index]
        
        context_text = " ".join(context_words)
        text_filtered_out.append(context_text)
        
    return text_filtered_out
            

keywords_counter = 0
start_time = time.time()
loaded_tt_df["words"] = loaded_tt_df["Text"].apply(lambda x: filter_keywords_context(x))
end_time = time.time()
print("Execution time: ", end_time-start_time)
loaded_tt_df = loaded_tt_df[["GOID", "Title", "words", "Date"]]
loaded_tt_df.to_csv("QTATITLE/tt_df_reduced.csv") # Execution time:  900

In [ ]:
new_loaded_tt_df = pd.read_csv("QTATITLE/tt_df_reduced.csv", index_col=0)
new_loaded_tt_df

In [ ]:
new_loaded_tt_df = pd.read_csv("QTATITLE/tt_df_reduced.csv", index_col=0)
new_loaded_tt_reduced = new_loaded_tt_df[["GOID", "words"]]
new_loaded_tt_reduced.to_csv("QTATITLE/tt_df_reduced_2.csv.xz", index=False, compression="xz")
new_loaded_tt_reduced

In [ ]:
df = pd.read_csv("QTATITLE/tt_df_reduced_2.csv.xz", compression="xz")
df

## Prepare Final Data Set for Analysis

Lastly, we construct our full dataset based on all the iterations that we have done before.

In [ ]:
import pandas as pd
import re
import time
import numpy as np

To do so, we merge the different datasets created before, such that we have a full dataset with indicators for all previous steps.

In [ ]:
full_df = pd.read_csv("extended.csv")
full_df = full_df[["GOID", "Title", "Date"]]
full_df

In [ ]:
title_contains = pd.read_csv("title_contains.csv", index_col=0)
title_contains

In [ ]:
words_df = pd.read_csv("tt_df_reduced_2.csv.xz", compression="xz")
based_on = pd.read_csv("cleaned_tt_df_download.csv", index_col=0)

words_based_df = pd.merge(words_df, based_on, how="inner", left_on="GOID", right_on="GOID")
words_based_df

In [ ]:
merge_1 = pd.merge(full_df, title_contains, how="inner", left_on="GOID", right_on="GOID")
merge_2 = pd.merge(merge_1, words_based_df, how="left", left_on="GOID", right_on="GOID")
merge_2

We also (1) create a column that holds all companies contained in the title and a column that contains all companies contained in the text...

In [ ]:
company_df = pd.read_csv("Companies.csv", encoding="cp1252")
company_df = company_df[company_df["Companies"] != "Target"]
company_df = company_df[company_df["Companies"] != "Visa"]

comp_df = company_df["Companies"].to_list()
pattern_comp = re.compile(r'\b(?:' + '|'.join(re.escape(company) for company in comp_df) + r')\b')


def contains_company_name(text, pattern):
    global keywords_counter
    keywords_counter += 1
    if keywords_counter%10000==0:
        print(keywords_counter)
    if text:
        matches = pattern.findall(text)
        return list(set(matches if matches else None))
    else:
        return None

start_time = time.time()
keywords_counter = 0
merge_2["company_in_title"] = merge_2["Title"].apply(lambda x: contains_company_name(x, pattern_comp))
end_time = time.time()
print("Execution time: ", end_time-start_time)

In [ ]:
company_df = pd.read_csv("Companies.csv", encoding="cp1252")
company_df = company_df[company_df["Companies"] != "Target"]
company_df = company_df[company_df["Companies"] != "Visa"]

comp_df = company_df["Companies"].to_list()
pattern_comp = re.compile(r'\b(?:' + '|'.join(re.escape(company) for company in comp_df) + r')\b')

def contains_company_name_2(text, pattern):
    global keywords_counter
    keywords_counter += 1
    if keywords_counter%10000==0:
        print(keywords_counter)
    if text and isinstance(text, str):
        matches = pattern.findall(text)
        if matches:
            return list(set(matches if matches else None))
    else:
        return None

start_time = time.time()
keywords_counter = 0
merge_2["company_in_text"] = merge_2["words"].apply(lambda x: contains_company_name_2(x, pattern_comp))
end_time = time.time()
print("Execution time: ", end_time-start_time)

...and (2) a column that contains all scandals contained in the title and a column that contains all scandals contained in the text.

In [ ]:
scandal_df = pd.read_csv("keywords.csv", index_col=False)
scand_df = scandal_df["keywords"].to_list()
pattern_string = '|'.join("(?P<{}>{})".format(re.sub(r'\W', '_', sca), re.escape(sca)) for sca in scand_df)
pattern_string_repl = pattern_string.replace("\.\*", ".*")
pattern_scandal = re.compile(r'\b(?:' + pattern_string_repl + r')\b', re.IGNORECASE)

    
def contains_scandal(text, pattern):
    global keywords_counter
    keywords_counter += 1
    if keywords_counter%10000==0:
        print(keywords_counter)
    if text:
        matches = [name for match in pattern.finditer(text) for name, value in match.groupdict().items() if value]
        if matches:
            return list(set(matches if matches else None))
    else:
        return None

start_time = time.time()
keywords_counter = 0
merge_2["scandal_in_title"] = merge_2["Title"].apply(lambda x: contains_scandal(x, pattern_scandal))
end_time = time.time()
print("Execution time: ", end_time-start_time)

In [ ]:
scandal_df = pd.read_csv("keywords.csv", index_col=False)
scand_df = scandal_df["keywords"].to_list()
pattern_string = '|'.join("(?P<{}>{})".format(re.sub(r'\W', '_', sca), re.escape(sca)) for sca in scand_df)
pattern_string_repl = pattern_string.replace("\.\*", ".*")
pattern_scandal = re.compile(r'\b(?:' + pattern_string_repl + r')\b', re.IGNORECASE)

def contains_scandal_2(text, pattern):
    global keywords_counter
    keywords_counter += 1
    if keywords_counter%10000==0:
        print(keywords_counter)
    if text and isinstance(text, str):
        matches = [name for match in pattern.finditer(text) for name, value in match.groupdict().items() if value]
        if matches:
            return list(set(matches if matches else None))
    else:
        return None

start_time = time.time()
keywords_counter = 0
merge_2["scandal_in_text"] = merge_2["words"].apply(lambda x: contains_scandal_2(x, pattern_scandal))
end_time = time.time()
print("Execution time: ", end_time-start_time)

In [ ]:
merge_2_unique = merge_2.drop_duplicates(subset='Title', keep="first")
merge_2_unique

## Merge in comparisons

Finally, we merge in all data from the validation, i.e. the data from doccano and the data from ChatGPT.

In [ ]:
gpt_comparison = pd.read_csv("GPT_comparing.csv", index_col=0)
gpt_comparison

In [ ]:
scandals_manual_comparison = pd.read_csv("scandals_manually_comparing.csv", index_col=0)
scandals_manual_comparison = scandals_manual_comparison.rename(columns={"text":"Title", "Unnamed: 2":"A", "Date":"B", "label":"scandals_manual","Comments":"C"})
scandals_manual_comparison = scandals_manual_comparison[["Title", "scandals_manual"]]
scandals_manual_comparison["scandals_manual"].value_counts()

In [ ]:
all_manual_comparison = pd.read_csv("all_manually_comparing.csv", index_col=0)
all_manual_comparison = all_manual_comparison .rename(columns={"Unnamed: 4":"scandals_all"})
all_manual_comparison = all_manual_comparison [["Title", "scandals_all"]]
all_manual_comparison

In [ ]:
merge_3 = pd.merge(merge_2_unique, gpt_comparison[["Title", "GPT"]], how="left", left_on="Title", right_on="Title")

In [ ]:
merge_4 = pd.merge(merge_3, scandals_manual_comparison, how="left", left_on="Title", right_on="Title")

In [ ]:
merge_5 = pd.merge(merge_4, all_manual_comparison, how="left", left_on="Title", right_on="Title")

In [ ]:
merge_5.to_csv("final_data_with_count_and_annotation.csv")

This merge also allows us to create various confusion matrices for title, text or both. The code can be easily adapted for a certain comparison, here we have the code for the GPT comparison and the code for a doccano comparison.

In [ ]:
xxxxx = pd.merge(merge_2_unique, gpt_comparison, how="inner", left_on="Title", right_on="Title")
xxxxx = xxxxx[["Textbased", "GPT", "Titlebased"]].fillna('N')
xxxxx["Textbased"] = xxxxx["Textbased"].apply(lambda x: "Y" if x==1.0 else "N")
xxxxx["Titlebased"] = xxxxx["Titlebased"].apply(lambda x: "Y" if x==1.0 else "N")
# xxxxx["Textbased"] = xxxxx["Textbased"] + xxxxx["Titlebased"]
# xxxxx["Textbased"] = xxxxx["Textbased"].apply(lambda x: "Y" if x=="YY" or x=="NY" or x=="YN" else "N")

cm = confusion_matrix(xxxxx["GPT"], xxxxx["Titlebased"], labels=['N', 'Y'])
cm

In [ ]:
xxxxx = pd.merge(merge_2_unique, scandals_manual_comparison, how="inner", left_on="Title", right_on="Title")
xxxxx = xxxxx[["Textbased", "scandals_manual", "Titlebased"]].fillna('N')
xxxxx["Titlebased"] = xxxxx["Titlebased"].apply(lambda x: "Y" if x==1.0 else "N")
#xxxxx["Titlebased"] = xxxxx["Titlebased"].apply(lambda x: "Y" if x==1.0 else "N")
#xxxxx["Textbased"] = xxxxx["Textbased"] + xxxxx["Titlebased"]
#xxxxx["Textbased"] = xxxxx["Textbased"].apply(lambda x: "Y" if x=="YY" or x=="NY" or x=="YN" else "N")

cm = confusion_matrix(xxxxx["scandals_manual"], xxxxx["Titlebased"], labels=['N', 'Y'])
cm

## Load Final Dataset

Now the data is ready for analysis...

In [ ]:
df = pd.read_csv("final_data_with_count_and_annotation.csv", index_col=0)
df

...or so we thought until we checked the results. We realized that there were some significant results for variations and abbreviations of company names, e.g. BofA and Bank of America both had a good amount of scandals. Thus, we are mapping the company spellings (for the most important, i.e., having the most results, ones) back to a single company name.

In [ ]:
company_mapping = {
    "Wal-Mart": ["Wal-Mart Stores", "Walmart"],
    "IBM": ["Intl. Business Machines", "International Business Machines"],
    "Citi": ["Citigroup"],
    "AIG": ["American Intl. Group", "American International Group"],
    "J.P. Morgan": ["JP Morgan", "JPMorgan Chase", "J.P. Morgan Chase & Co.", "JPM"],
    "MS": ["Morgan Stanley", "MS"],
    "McDonald's": ["McDonald", "McDonalds"],
    "Verizon": ["Verizon Communications"],
    "Sprint": ["Sprint Nextel"],
    "Dell": ["Dell Technologies"],
    "Exxon": ["Exxon Mobil"],
    "Lehman": ["Lehman Brothers Hldgs.", "Lehman Brothers"],
    "BofA": ["Bank of America Corp.", "Bank of America"],
    "Travelers": ["Travelers Cos."],
    "Cisco": ["Cisco Systems"],
    "Chevron": ["ChevronTexaco"],
    "Blackstone": ["Blackstone Group"],
    "UPS": ["United Parcel Service"],
    "Coca-Cola": ["Coca Cola"],
    "DuPont": ["DuPont de Nemours"],
    "CVS": ["CVS Health", "CVS Caremark"],
    "Pepsi": ["PepsiCo"],
    "SBC": ["SBC Communications"],
    "Macy's": ["Macy"],
    "General Motors": ["GM"],
    "DirecTV": ["DirecTV Group"],
    "Duke": ["Duke Energy"],
    "American Express": ["AmEx"],
    "MasterCard": ["Mastercard"],
    "Walmart": ["Wal-Mart", "Wal-Mart Stores"],
    "Goldman": ["Goldman Sachs Group", "GS"],
    "Disney": ["Walt Disney"],
    "Apple": ["Apple Computer"],
    "Google": ["Alphabet"],
    "Procter & Gamble": ["P&G"],
    "Johnson & Johnson": ["J&J"],
    "ToysRUs": ["Toys R Us"]
}

In [ ]:
def replace_company_names(companies, mapping):
    reversed_mapping = {alias: key for key, aliases in mapping.items() for alias in aliases}
    return str([reversed_mapping.get(company, company) for company in eval(companies)])

df["company_in_title"] = df["company_in_title"].apply(lambda x: replace_company_names(x, company_mapping))

In [ ]:
def replace_company_names(companies, mapping):
    reversed_mapping = {alias: key for key, aliases in mapping.items() for alias in aliases}
    if isinstance(companies, str):
        return str([reversed_mapping.get(company, company) for company in eval(companies)])
    else:
        return pd.NA

df["company_in_text"] = df["company_in_text"].apply(lambda x: replace_company_names(x, company_mapping))
df['company_in_text'] = df['company_in_text'].apply(lambda x: np.nan if pd.isna(x) else x)

And with that, we really arrive at the final dataset that can be used for analysis.

In [ ]:
df.to_csv("FINAL_DATASET_FIX.csv")
df

In [ ]:
loaded_df = pd.read_csv("final_data_with_count_and_annotation.csv", index_col=0)
loaded_df

In [ ]:
loaded_df